In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.logging import InteractiveLogger
from avalanche.evaluation.metrics import accuracy_metrics, loss_metrics
from avalanche.training.strategies import BaseStrategy
from avalanche.training.plugins import EvaluationPlugin,ReplayPlugin
from torchvision.transforms import Compose, ToTensor, Normalize, RandomCrop
%matplotlib inline

In [2]:
train_transform = Compose([
    RandomCrop(28, padding=4),
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

test_transform = Compose([
    ToTensor(),
    Normalize((0.1307,), (0.3081,))
])

In [3]:
mnist_train = torchvision.datasets.MNIST(
    'E:/Datasets/', train=True, download=True, transform=train_transform
)
mnist_test = torchvision.datasets.MNIST(
    'E:/Datasets/', train=False, download=True, transform=test_transform
)

In [4]:
scenario = nc_benchmark(
    mnist_train, mnist_test, n_experiences=10, shuffle=True, seed=1234,
    task_labels=True
)

train_stream = scenario.train_stream

for experience in train_stream:
    t = experience.task_label
    exp_id = experience.current_experience
    training_dataset = experience.dataset
    print('Task {} batch {} -> train'.format(t, exp_id))
    print('This batch contains', len(training_dataset), 'patterns')
    print('This batch contains',experience.classes_in_this_experience)
    print('Classes seen:',experience.classes_seen_so_far)

Task 0 batch 0 -> train
This batch contains 5421 patterns
This batch contains [5]
Classes seen: [5]
Task 1 batch 1 -> train
This batch contains 5842 patterns
This batch contains [4]
Classes seen: [4, 5]
Task 2 batch 2 -> train
This batch contains 5851 patterns
This batch contains [8]
Classes seen: [8, 4, 5]
Task 3 batch 3 -> train
This batch contains 5958 patterns
This batch contains [2]
Classes seen: [8, 2, 4, 5]
Task 4 batch 4 -> train
This batch contains 5918 patterns
This batch contains [6]
Classes seen: [2, 4, 5, 6, 8]
Task 5 batch 5 -> train
This batch contains 5949 patterns
This batch contains [9]
Classes seen: [2, 4, 5, 6, 8, 9]
Task 6 batch 6 -> train
This batch contains 6742 patterns
This batch contains [1]
Classes seen: [1, 2, 4, 5, 6, 8, 9]
Task 7 batch 7 -> train
This batch contains 6265 patterns
This batch contains [7]
Classes seen: [1, 2, 4, 5, 6, 7, 8, 9]
Task 8 batch 8 -> train
This batch contains 5923 patterns
This batch contains [0]
Classes seen: [0, 1, 2, 4, 5, 6, 7

In [5]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 20, 5, 1)
        self.conv2 = nn.Conv2d(20, 50, 5, 1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2, 2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)
globalModel=Net()

In [6]:
def trainAndEvaluate(strategy,scenario):
    train_stream=scenario.train_stream
    test_stream=scenario.test_stream
    for experience in train_stream:
        t = experience.task_label
        exp_id = experience.current_experience
        training_scenario = experience.scenario
        print('Task {} batch {} -> train'.format(t, exp_id))
        print('This batch contains',experience.classes_in_this_experience)
        print('Classes seen:',experience.classes_seen_so_far)
        strategy.train(experience)
        strategy.eval(test_stream)


In [7]:
interactive_logger=InteractiveLogger()

eval_plugin = EvaluationPlugin(
accuracy_metrics(minibatch=True, epoch=True, experience=True, stream=True),
loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),benchmark=scenario,
loggers=[interactive_logger])

In [8]:
replay = ReplayPlugin(mem_size=100)

strategy=BaseStrategy(
    globalModel,optim.Adam(globalModel.parameters()),train_mb_size=64,eval_mb_size=32,evaluator=eval_plugin,plugins=[replay]
)
trainAndEvaluate(strategy,scenario)

Task 0 batch 0 -> train
This batch contains [5]
Classes seen: [5]
-- >> Start of training phase << --
-- Starting training on experience 0 (Task 0) from train stream --
0it [00:00, ?it/s]

C:\Users\Ashwin\AppData\Local\Temp/ipykernel_15004/2922225201.py:7: UserWarning: Using self.scenario is deprecated in Experience. Consider using self.benchmark instead.
  training_scenario = experience.scenario


  6%|████▉                                                                              | 5/85 [00:01<01:31,  1.14s/it]

c:\users\ashwin\miniconda3\lib\site-packages\torch\nn\functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ..\c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


100%|██████████████████████████████████████████████████████████████████████████████████| 85/85 [00:07<00:00, 11.89it/s]
Epoch 0 ended.
	Loss_Epoch/train_phase/train_stream/Task000 = 0.0481
	Loss_MB/train_phase/train_stream/Task000 = 0.0000
	Top1_Acc_Epoch/train_phase/train_stream/Task000 = 0.9882
	Top1_Acc_MB/train_phase/train_stream/Task000 = 1.0000
-- >> End of training phase << --
-- >> Start of eval phase << --
-- Starting eval on experience 0 (Task 0) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 86.12it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.0000
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 1.0000
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 86.34it/s]
> Eval on experience 1 (Task 1) from test s

...

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:00<00:00, 86.60it/s]
> Eval on experience 0 (Task 0) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task000/Exp000 = 0.1618
	Top1_Acc_Exp/eval_phase/test_stream/Task000/Exp000 = 0.9462
-- Starting eval on experience 1 (Task 1) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 89.32it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 0.1247
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.9481
-- Starting eval on experience 2 (Task 2) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 83.78it/s]
> Eval on experience 2 (Task 2) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task002/Exp002 = 0.3537
	Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002 = 0

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 95.49it/s]
> Eval on experience 8 (Task 8) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task008/Exp008 = 21.1807
	Top1_Acc_Exp/eval_phase/test_stream/Task008/Exp008 = 0.0000
-- Starting eval on experience 9 (Task 9) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 77.88it/s]
> Eval on experience 9 (Task 9) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task009/Exp009 = 28.3821
	Top1_Acc_Exp/eval_phase/test_stream/Task009/Exp009 = 0.0000
-- >> End of eval phase << --
	Loss_Stream/eval_phase/test_stream/Task000 = 0.1721
	Loss_Stream/eval_phase/test_stream/Task001 = 0.1203
	Loss_Stream/eval_phase/test_stream/Task002 = 0.3060
	Loss_Stream/eval_phase/test_stream/Task003 = 0.0648
	Loss_Stream/eval_phase/test_stream/Task004 = 0.0235
	Loss_Stream/eval_phase/test_stream/Task005 = 22.8826

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 88.30it/s]
> Eval on experience 1 (Task 1) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task001/Exp001 = 0.5369
	Top1_Acc_Exp/eval_phase/test_stream/Task001/Exp001 = 0.8717
-- Starting eval on experience 2 (Task 2) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 86.94it/s]
> Eval on experience 2 (Task 2) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task002/Exp002 = 0.2847
	Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002 = 0.9230
-- Starting eval on experience 3 (Task 3) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 85.28it/s]
> Eval on experience 3 (Task 3) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task003/Exp003 = 0.1672
	Top1_Acc_Exp/eval_phase/test_stream/Task003/Exp003 = 0

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [00:00<00:00, 86.72it/s]
> Eval on experience 9 (Task 9) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task009/Exp009 = 28.8290
	Top1_Acc_Exp/eval_phase/test_stream/Task009/Exp009 = 0.0000
-- >> End of eval phase << --
	Loss_Stream/eval_phase/test_stream/Task000 = 0.2898
	Loss_Stream/eval_phase/test_stream/Task001 = 1.0130
	Loss_Stream/eval_phase/test_stream/Task002 = 0.5418
	Loss_Stream/eval_phase/test_stream/Task003 = 0.3807
	Loss_Stream/eval_phase/test_stream/Task004 = 0.0816
	Loss_Stream/eval_phase/test_stream/Task005 = 0.3231
	Loss_Stream/eval_phase/test_stream/Task006 = 0.1305
	Loss_Stream/eval_phase/test_stream/Task007 = 0.0169
	Loss_Stream/eval_phase/test_stream/Task008 = 27.1485
	Loss_Stream/eval_phase/test_stream/Task009 = 28.8290
	Top1_Acc_Stream/eval_phase/test_stream/Task000 = 0.9395
	Top1_Acc_Stream/eval_phase/test_stream/Task001 = 0.8218
	Top1_Acc_Stream/eval_phase/t

100%|██████████████████████████████████████████████████████████████████████████████████| 31/31 [00:00<00:00, 89.27it/s]
> Eval on experience 2 (Task 2) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task002/Exp002 = 0.2999
	Top1_Acc_Exp/eval_phase/test_stream/Task002/Exp002 = 0.9199
-- Starting eval on experience 3 (Task 3) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 33/33 [00:00<00:00, 83.56it/s]
> Eval on experience 3 (Task 3) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task003/Exp003 = 1.7007
	Top1_Acc_Exp/eval_phase/test_stream/Task003/Exp003 = 0.6812
-- Starting eval on experience 4 (Task 4) from test stream --
100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 88.21it/s]
> Eval on experience 4 (Task 4) from test stream ended.
	Loss_Exp/eval_phase/test_stream/Task004/Exp004 = 0.5081
	Top1_Acc_Exp/eval_phase/test_stream/Task004/Exp004 = 0